In [ ]:
# Script EAPV completo - tratamento dos dados

import pandas as pd
import datetime
import numpy as np
from unidecode import unidecode

pd.options.mode.chained_assignment = None

print("Inicio: " + datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

# Função remover acentos
def strip_accents(text):
    return str(unidecode(text))

def calc_idade(value, default, *types):
    for t in types:
        try:
            return t(value)
        except (ValueError, TypeError):
            continue
    return default

# Dados nome do municipio e CRS
data_mun_crs = pd.read_csv('data_mun_crs.csv', sep=';', encoding='cp1252')
data_mun_crs.columns = [x.upper() for x in list(data_mun_crs.columns.values)]
data_mun_crs['MUNICIPIO DE RESIDENCIA'] = data_mun_crs.apply(lambda e: strip_accents(e['MUNICIPIO DE RESIDENCIA']), axis=1)

# Dados SIVEP - confirmados COVID-19 - do painel BI SES
data_conf_painel = pd.read_csv('data_conf_painel.csv', sep=';', encoding='cp1252')
data_conf_painel.columns = [x.upper() for x in list(data_conf_painel.columns.values)]
#data_conf_painel['DATA SINTOMAS'] = data_conf_painel['DATA SINTOMAS'].dt.date
data_conf_painel['DATA SINTOMAS'] = pd.to_datetime(data_conf_painel['DATA SINTOMAS'])

# Dados EAPV
data_eapv = pd.read_excel('eapv.xlsx', dtype=str)
data_eapv = data_eapv[['Número da Notificação', 'Data da Notificação', 'Estado de Residência', 'Sexo', 'CPF', 'Município de Residência',
                       'Data de Nascimento', 'Nome Completo da Mãe', 'Nome Completo', 'É profissional de saúde?', 'Raça/Cor', 'Comunidade/Povo Tradicional',
                       'Relação imunobiológico ao evento adverso', 'Nome do Fabricante', 'Imunobiológico (vacina)', 'Código Imunobiológico',
                       'Lote', 'Dose', 'Data da aplicação', 'Descrição do caso ', 'Reação / evento adverso', 'Código Evento Adverso',
                       'Data de início', 'Tipo de Evento', 'Classificação de gravidade', 'Gravidade', 'Desfecho (evolução do caso)',
                       'Data Desfecho', 'Gestante no momento da vacinação?', 'Doenças (CID10)', 'Descricao detalhada do Evento Adverso',
                       'Houve atendimento médico?', 'Tipo de Atendimento', 'Observações complementares', 'CNES Estabelecimento de saúde',
                       'Nome do exame', 'Resultado do exame', 'Data da realização do exame', 'Diagnóstico (CID-10)', 'Grave?',
                       'Evento adverso', 'Tipo', 'Diagnóstico', 'Causalidade', 'Conduta', 'Informe a conduta', 'Comentários', 'Data Encerramento']]

# Colunas em Maiúsculo, sem acentos e sem espaços nas bordas
data_eapv.columns = [strip_accents(x.upper()).strip() for x in list(data_eapv.columns.values)]


# Tratamento dos dados
data_eapv = data_eapv.drop_duplicates()

imunobiologico = ['oxford', 'astrazeneca', 'fiocruz', 'pfizer', 'butantan', 'sinovac', 'covid', 'janssen']
data_eapv = data_eapv[data_eapv['IMUNOBIOLOGICO (VACINA)'].str.contains('(?i)' + '|'.join(imunobiologico), na = False)]

# Arquivo contendo somente vacinas COVID-19
#data_eapv.to_excel('Relatorios/imunobiologico.xlsx', index=False)

data_eapv['NOME COMPLETO'] = data_eapv.apply(lambda e: strip_accents(e['NOME COMPLETO'].upper().strip()), axis=1)
data_eapv['MUNICIPIO DE RESIDENCIA'] = data_eapv.apply(lambda e: strip_accents(e['MUNICIPIO DE RESIDENCIA']), axis=1)
data_eapv = data_eapv.merge(data_mun_crs, how='left')

data_eapv['CPF'] = data_eapv['CPF'].str.replace('.', '').str.replace('-', '')
data_eapv['DATA DE NASCIMENTO'] = pd.to_datetime(data_eapv['DATA DE NASCIMENTO'], errors='coerce', format='%d/%m/%Y').dt.date
#Data da aplicação não necessariamente está na mesma ordem a D1 e depois a D2, essas colunas só são utilizadas para verificação da covid em 14 dias
data_eapv['DATA DA APLICACAO D1'] = pd.to_datetime(data_eapv['DATA DA APLICACAO'].str.split(', ',expand=True)[0], errors='coerce', format='%d/%m/%Y').dt.date
data_eapv['DATA DA APLICACAO D2'] = pd.to_datetime(data_eapv['DATA DA APLICACAO'].str.split(', ',expand=True)[1], errors='coerce', format='%d/%m/%Y').dt.date

#Calcula a Idade e aplica inconsistente se < 18 e > 107 nas faixas etárias
data_eapv['IDADE'] = data_eapv.apply(lambda e: (np.floor((e['DATA DA APLICACAO D1'] - e['DATA DE NASCIMENTO']).days/365.25)).astype(int), axis=1)
data_eapv['FAIXA ETARIA'] = 'Inconsistente'
data_eapv.loc[(data_eapv['IDADE'] >= 18) & (data_eapv['IDADE'] <= 19), 'FAIXA ETARIA'] = '18-19'
data_eapv.loc[(data_eapv['IDADE'] >= 20) & (data_eapv['IDADE'] <= 24), 'FAIXA ETARIA'] = '20-24'
data_eapv.loc[(data_eapv['IDADE'] >= 25) & (data_eapv['IDADE'] <= 29), 'FAIXA ETARIA'] = '25-29'
data_eapv.loc[(data_eapv['IDADE'] >= 30) & (data_eapv['IDADE'] <= 34), 'FAIXA ETARIA'] = '30-34'
data_eapv.loc[(data_eapv['IDADE'] >= 35) & (data_eapv['IDADE'] <= 39), 'FAIXA ETARIA'] = '35-39'
data_eapv.loc[(data_eapv['IDADE'] >= 40) & (data_eapv['IDADE'] <= 44), 'FAIXA ETARIA'] = '40-44'
data_eapv.loc[(data_eapv['IDADE'] >= 45) & (data_eapv['IDADE'] <= 49), 'FAIXA ETARIA'] = '45-49'
data_eapv.loc[(data_eapv['IDADE'] >= 50) & (data_eapv['IDADE'] <= 54), 'FAIXA ETARIA'] = '50-54'
data_eapv.loc[(data_eapv['IDADE'] >= 55) & (data_eapv['IDADE'] <= 59), 'FAIXA ETARIA'] = '55-59'
data_eapv.loc[(data_eapv['IDADE'] >= 60) & (data_eapv['IDADE'] <= 64), 'FAIXA ETARIA'] = '60-64'
data_eapv.loc[(data_eapv['IDADE'] >= 65) & (data_eapv['IDADE'] <= 69), 'FAIXA ETARIA'] = '65-69'
data_eapv.loc[(data_eapv['IDADE'] >= 70) & (data_eapv['IDADE'] <= 74), 'FAIXA ETARIA'] = '70-74'
data_eapv.loc[(data_eapv['IDADE'] >= 75) & (data_eapv['IDADE'] <= 79), 'FAIXA ETARIA'] = '75-79'
data_eapv.loc[(data_eapv['IDADE'] >= 80) & (data_eapv['IDADE'] <= 107), 'FAIXA ETARIA'] = '80+'

print(f'Banco completo somente com vacinas COVID-19: {data_eapv.shape}')

# Relatório 1 - Erro de Imunizacao
df_erro_imunizacao = data_eapv.loc[data_eapv['TIPO DE EVENTO'].str.contains("(?i)erro de imunização")]
data_eapv = data_eapv.loc[~data_eapv['NUMERO DA NOTIFICACAO'].isin(df_erro_imunizacao['NUMERO DA NOTIFICACAO'])]
print(f'Banco Erros de Imunização: {df_erro_imunizacao.shape}')

#Excluir faixas etárias Inconsistentes 
data_eapv = data_eapv.loc[data_eapv['FAIXA ETARIA'] != 'Inconsistente']
print(f'Banco EAPV sem EI e faixas etárias inconsistentes: {data_eapv.shape}')

# Normaliza as colunas com texto de campo aberto para remover acentos e deixar tudo minusculo
cols = ['DESCRICAO DETALHADA DO EVENTO ADVERSO', 'DESCRICAO DO CASO', 'OBSERVACOES COMPLEMENTARES', 'NOME DO EXAME', 'RESULTADO DO EXAME',
        'EVENTO ADVERSO', 'GRAVIDADE', 'DESFECHO (EVOLUCAO DO CASO)']
data_eapv[cols] = data_eapv[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower())
data_eapv['REACAO / EVENTO ADVERSO'] = data_eapv.apply(lambda x: strip_accents(x['REACAO / EVENTO ADVERSO']), axis=1)

# Relatório 2 -EAPV com Infecção por COVID
#cria a coluna com o nome do exame simplificada
data_eapv['NOME_EXAME'] = pd.Series(dtype=str)
data_eapv.loc[data_eapv['NOME DO EXAME'].str.contains('pcr', na = False), 'NOME_EXAME'] = 'pcr'
data_eapv.loc[data_eapv['NOME DO EXAME'].str.contains('antigeno', na = False), 'NOME_EXAME'] = 'antigeno'
data_eapv.loc[data_eapv['NOME DO EXAME'].str.contains('swab', na = False), 'NOME_EXAME'] = 'covid'

#cria a coluna com o resultado do exame simplificada
data_eapv['RESULTADO_EXAME'] = pd.Series(dtype=str)
data_eapv['RESULTADO DO EXAME'] = data_eapv['RESULTADO DO EXAME'].str.replace('nao detectavel', 'negativo')\
                                                                 .str.replace('nao detectado', 'negativo')\
                                                                 .str.replace('nao reagente', 'negativo')
resultado_exame = ['positivo', 'detectavel', 'detectado', 'reagente']
data_eapv.loc[data_eapv['RESULTADO DO EXAME'].str.contains('|'.join(resultado_exame), na = False), 'RESULTADO_EXAME'] = 'positivo'

#cria a coluna covid_positivo com nome_exame + resultado_exame ou outras condições
data_eapv['COVID_POSITIVO'] = pd.Series(dtype=str)
data_eapv.loc[(((data_eapv['NOME_EXAME'] == 'pcr') | (data_eapv['NOME_EXAME'] == 'antigeno') | (data_eapv['NOME_EXAME'] == 'covid'))  &
              (data_eapv['RESULTADO_EXAME']=='positivo')) | (data_eapv['EVENTO ADVERSO'].str.contains('(?i)covid-19', na = False)) |
              (data_eapv['DIAGNOSTICO'].str.contains('(?i)b342|u071|u072|b972|u04', na=False)) |
              (data_eapv['REACAO / EVENTO ADVERSO'].str.contains('(?i)teste de covid-19 por pcr positivo', na=False)), 'COVID_POSITIVO'] = 'covid+'

print(f'Banco EAPV: {data_eapv.shape}')

df_covid = data_eapv[data_eapv['COVID_POSITIVO'] == 'covid+']
print(f'Banco COVID: {df_covid.shape}')

data_eapv = data_eapv.loc[data_eapv['COVID_POSITIVO'] != 'covid+']
print(f'Banco EAPV sem COVID: {data_eapv.shape}')


# Relatório 3 - Base limpa com óbito
data_eapv['OBITO'] = 'nao'
obito = ['obito', 'falec', '\\bmorte\\b'] #Pesquisa por toda palavra 'morte'

data_eapv.loc[(data_eapv['DESCRICAO DETALHADA DO EVENTO ADVERSO'].str.contains('|'.join(obito), na = False)) |
              (data_eapv['DESCRICAO DO CASO'].str.contains('|'.join(obito), na = False)) |
              (data_eapv['GRAVIDADE'].str.contains('ocasione o obito', na = False)), 'OBITO'] = 'provavel'

data_eapv.loc[(data_eapv['OBSERVACOES COMPLEMENTARES'].str.contains('|'.join(obito), na = False)) |
              (data_eapv['EVENTO ADVERSO'].str.contains('|'.join(obito), na = False)) |
              (data_eapv['REACAO / EVENTO ADVERSO'].str.contains('(?i)' + '|'.join(obito), na = False)) |
              (data_eapv['DESFECHO (EVOLUCAO DO CASO)'].str.contains('|'.join(obito), na = False)), 'OBITO'] = 'sim'


# Relatório 4 - Reações adversas
df_reacao = pd.DataFrame()
cont = 0
for index, row in data_eapv.iterrows():
    rea_eve_adverso = row['REACAO / EVENTO ADVERSO'].split(', ')
    cod_eve_adverso = row['CODIGO EVENTO ADVERSO'].split(', ')
    gravidade = row['CLASSIFICACAO DE GRAVIDADE'].split(', ')

    #Trata os casos em que tem no nome uma vírgula, como 'Mialgia e miosite, não especificadas'
    pos = 0
    if len(rea_eve_adverso) != len(cod_eve_adverso):
        i = 0
        for rea in rea_eve_adverso:
            #Verifica se a primeira letra não é maiúscula
            if rea[0].isupper() == False:
                rea_eve_adverso[pos] += ', ' + rea
                rea_eve_adverso[i] = ''
            else:
                pos = i
            i += 1
        #exclui no vetor as colunas vazias
        rea_eve_adverso = [x for x in rea_eve_adverso if x]

    i = 0
    dados = {}
    for rea in rea_eve_adverso:
        # Trata os dados e concatena depois no dataframe
        dados.setdefault("NUMERO DA NOTIFICACAO", []).append(row['NUMERO DA NOTIFICACAO'])
        dados.setdefault("CODIGO IMUNOBIOLOGICO", []).append(row['CODIGO IMUNOBIOLOGICO'])
        dados.setdefault("IMUNOBIOLOGICO (VACINA)", []).append(row['IMUNOBIOLOGICO (VACINA)'])
        dados.setdefault("DOSE", []).append(row['DOSE'])
        dados.setdefault("DATA DA APLICACAO", []).append(row['DATA DA APLICACAO'])
        dados.setdefault("SEXO", []).append(row['SEXO'])
        dados.setdefault("IDADE", []).append(row['IDADE'])
        dados.setdefault("FAIXA ETARIA", []).append(row['FAIXA ETARIA'])
        dados.setdefault("CAUSALIDADE", []).append(row['CAUSALIDADE'])
        dados.setdefault("REACAO / EVENTO ADVERSO", []).append(rea)
        dados.setdefault("CODIGO EVENTO ADVERSO", []).append(cod_eve_adverso[i])
        dados.setdefault("CLASSIFICACAO DE GRAVIDADE", []).append(gravidade[i])

        i += 1

    df_reacao = pd.concat([df_reacao, pd.DataFrame(dados)])

    
#Exporta relatórios
df_erro_imunizacao.to_csv('Relatorios/erro_imunizacao.csv', sep=';', encoding='utf-8-sig', index=False)
df_covid.to_csv('Relatorios/infeccao_covid.csv', sep=';', encoding='utf-8-sig', index=False)
data_eapv.to_csv('Relatorios/base_limpa.csv', sep=';', encoding='utf-8-sig', index=False)
df_reacao.to_csv('Relatorios/evento_adverso.csv', sep=';', encoding='utf-8-sig', index=False)

print('ok')
print("Término: " + datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))